# Import Libraries and Data

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import datetime
from sklearn import preprocessing
#import os
%matplotlib inline

df = pd.read_csv('/Users/jasongangel/Downloads/AIS/data/WorkOrder7538249_SAIS_2016-01.csv', usecols=['MMSI','PERIOD', 'Latitude', 'Longitude','SPEED_KNOTS', 'COG_DEG', 'SHIP_AND_CARGO_TYPE','DRAUGHT'], na_values ='None',low_memory=False, parse_dates=['PERIOD'])

df = df.rename(columns={'Latitude': 'LAT', 'Longitude': 'LON', 'SHIP_AND_CARGO_TYPE':'SHIP_TYPE'})

# Datashader Geospatial Plot

In [2]:
#DATASHADER Plot of LAT/LONGS
import datashader as ds
import datashader.transfer_functions as tf
import dask.dataframe as dd
dd = dd.read_csv('/Users/jasongangel/Downloads/AIS/data/Clean/Clean-01_16*.csv').persist()

canvas = ds.Canvas(plot_width=500, plot_height=300, 
                   x_range=(-8,8), y_range=(-8,8), 
                   x_axis_type='linear', y_axis_type='linear')

%time tf.shade(ds.Canvas().points(dd,'LONG','LAT'))

OSError: /Users/jasongangel/Downloads/AIS/data/Clean/Clean-01_16*.csv resolved to no files

#  Initial Data Look

In [ ]:
df.head(5)

In [ ]:
#Dataframe Info (SHIP_TYPE and DRAUGHT contain bad str values in some cells thus they are detected as object types)
df.info(memory_usage='deep')

In [ ]:
# Stats : COG_DEG should not exceed 360 and SPEED_KNOTS should not exceed 35kts
df.describe()

In [ ]:
#Percentage of Rows w/ Missing Data
print((df.isnull().sum().sum()/df.shape[0])*100,'%')

In [ ]:
#Missing data per column
df.isnull().sum()

# Data clean 

In [ ]:
#Copy df
df1 = df.copy()

MMSI

In [ ]:
#Number of individual Vessels
print(df1['MMSI'].nunique(),'Individual Vessels')

In [ ]:
#Remove invalid MMSIs and sort by clean MMSIs
df1=df[(df['MMSI'] > 200000000)]

#Create Country Column from first three digits of MMSI
df1['MMSI'] = df1['MMSI'].astype(str)
df1['COUNTRY'] = df1['MMSI'].str.slice(start=0, stop=3)

#Drop MMSI Column
df1 = df1.drop(['MMSI'],axis=1)

SPEED_KNOTS

In [ ]:
#Drop null SPEED_KNOTS values
df1.dropna(how='any', axis=0, subset=['SPEED_KNOTS'], inplace=True)

#Drop Rows with speed over 35kts
df1=df1[df1.SPEED_KNOTS <= 35]

COG_DEG

In [ ]:
#Remove any COG_DEG values above 360
df1 = df1[(df1['COG_DEG'] <= 360)]

#Forward fill COG_DEG Null values 
df1['COG_DEG'] = df1['COG_DEG'].fillna(method='ffill')

#Remove excess float precision
df1['COG_DEG'] = df1['COG_DEG'].astype(int)

SHIP_TYPE and DRAUGHT

In [ ]:
#Fill DRAUGHT null values with 0
df1['DRAUGHT']= df1['DRAUGHT'].fillna(0)

#Cast SHIP_TYPE  and DRAUGHT to numeric type and mark incompatable values as NAN
df1[['SHIP_TYPE','DRAUGHT']] = df1[['SHIP_TYPE','DRAUGHT']].apply(pd.to_numeric,errors='coerce',axis=0)

#Filter out vessel types other than Cargo and Tanker vessesls
df1 = df1[(df1['SHIP_TYPE'] >= 70) & (df1['SHIP_TYPE'] <= 89)]

#Cast SHIP_TYPE as a str
df1['SHIP_TYPE'] = df1['SHIP_TYPE'].astype(str)

#Remove second digit of SHIP_TYPE (has no significance)
df1['SHIP_TYPE'] = df1['SHIP_TYPE'].str.slice(start=0, stop=1)

#Change Back to int
df1[['SHIP_TYPE']] = df1['SHIP_TYPE'].astype(int)

#Display count of unique vessel types
df1.SHIP_TYPE.value_counts()

Sort by Period

In [ ]:
#Downcast float columns
cols = ['SPEED_KNOTS','COG_DEG', 'DRAUGHT','LAT','LON']
df1[cols] = df1[cols].round(2).apply(pd.to_numeric, downcast='float', axis=0)

#Downcast int columns
cols = ['SHIP_TYPE', 'COUNTRY']
df1[cols] = df1[cols].apply(pd.to_numeric, downcast='unsigned', axis=0)

#Sort by PERIOD
df1 = df1.sort_values('PERIOD')

# Clean Data Analysis

In [ ]:
df1.head(10)

In [ ]:
df1.tail(10)

In [ ]:
df1.info(memory_usage='deep')

In [ ]:
df1.describe(include='all')

In [ ]:
#Check again for null values
df1.isnull().sum()

In [ ]:
#SPEED_KNOTs Hist
plt.figure(figsize=(10,4))
sns.distplot(df1['SPEED_KNOTS'],kde=True,bins=30)

In [ ]:
#DRAUGHT Hist
plt.figure(figsize=(10,4))
sns.distplot(df1['DRAUGHT'],kde=True,bins=30)

In [ ]:
# Box Plot of SPEED_KNOTS by Cargo(7) and Tanker(8) vessels
plt.figure(figsize=(15,7))
sns.boxplot(x='SHIP_TYPE', y='SPEED_KNOTS',data=df1)

In [ ]:
#Box Plot Count by SHIP_TYPE
plt.figure(figsize=(10,7))
sns.countplot(x='SHIP_TYPE',data=df1)

# Export

In [ ]:
#import csv
#df1.to_csv('/Users/jasongangel/Downloads/AIS/data/Clean/Clean-01_16*.csv', index=False, encoding='utf-8')
#quoting=csv.QUOTE_NONE

In [ ]:
#from pandas import HDFStore,DataFrame

# create (or open) an hdf5 file and opens in append mode
#hdf = HDFStore('AIS_ALL.h5')

# put the dataset in the storage
#hdf.put('d1', df1, format='table', data_columns=True)

#hdf.append()
        
#hdf.close() # closes the file

In [ ]:
#hdf = pd.read_hdf('AIS_ALL.h5', key='d1', mode='r')
#hdf.head()